### Hands-on Exploration of GNN
---

Before diving into GCN, I should get a firm understanding of GNN in general, by which I should be able to replicate the work of simply architectures.

Remember, the purpose of this project is to learn how to utilize the power of graph structures, which may generate stellar power in the future.

Resources:
* [very hands-on tutorial](https://towardsdatascience.com/hands-on-graph-neural-networks-with-pytorch-pytorch-geometric-359487e221a8)
* PyG Libaray
* PyTorch Geometric Library

#### PyTorch Geometric Basics
---

In [1]:
import torch
from torch_geometric.data import Data

In [2]:
x = torch.tensor([[2,1], [5,6], [3,7], [12,0]], dtype=torch.float) # features at each node
y = torch.tensor([0, 1, 0, 1], dtype=torch.float) # targets

# first list: index of source nodes
# second list: index of target nodes
edge_index = torch.tensor([[0, 1, 2, 0, 3],
                           [1, 0, 1, 3, 2]], dtype=torch.long)

data = Data(x=x, y=y, edge_index=edge_index)

In [3]:
# or we can formulate edges as

edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [2, 1],
                           [0, 3],
                           [3, 2]], dtype=torch.long)

data = Data(x=x, y=y, edge_index=edge_index.t().contiguous())

#### Datasets

In [4]:
from torch_geometric.datasets import TUDataset

In [5]:
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
len(dataset)

Extracting /tmp/ENZYMES/ENZYMES/ENZYMES.zip
Processing...
Done!


600

In [6]:
dataset.num_classes

6

In [7]:
dataset.num_node_features

3

In [8]:
dataset.num_edge_features

0

In [9]:
train_dataset = dataset[:540]
test_dataset = dataset[540:]

In [10]:
dataset = dataset.shuffle()
dataset

ENZYMES(600)

In [11]:
# Cora
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [12]:
dataset.num_classes, dataset.num_node_features

(7, 1433)

In [13]:
dataset.data

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

In [14]:
data = dataset.data
print(data.train_mask.sum(), data.val_mask.sum(), data.test_mask.sum())

tensor(140) tensor(500) tensor(1000)


#### Batching
---

In [15]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader
from torch_scatter import scatter_mean

In [16]:
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [17]:
for i, batch in enumerate(loader):
    batch
i, batch

(18, Batch(batch=[829], edge_index=[2, 3008], x=[829, 21], y=[24]))

In [18]:
# aggregate node features
for data in loader:
    data.num_graphs
    x = scatter_mean(data.x, data.batch, dim=0)
    x.size()

In [19]:
x.size()

torch.Size([24, 21])

In [20]:
data

Batch(batch=[625], edge_index=[2, 2422], x=[625, 21], y=[24])

#### Learning on Graphs
---

In [21]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [22]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [23]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        
        return F.log_softmax(x, dim=1)

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [25]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct/data.test_mask.sum().item()

In [26]:
acc

0.798

#### Message Passing
---

In [27]:
# Implementing GCN from scratch

import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

As shown in the [tutorial](https://pytorch-geometric.readthedocs.io/en/latest/notes/create_gnn.html), a GCN Layer can be divided into the following components:
1. Add self-loop to the adj matrix.
2. Linear transform the node feature matrix.
3. Compute normalization coeffs.
4. Normalize node features in $\phi$.
5. Sum up neighboring nodes features (add).
6. Return new node embedding

In [28]:
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')
        self.lin = torch.nn.Linear(in_channels, out_channels)
        
    def forward(self, x, edge_index):
        # x.shape = (N, in_channels)
        # edge_index.shape = (2, E)
        
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))  # step 1
        x = self.lin(x)  # step 2
        row, col = edge_index  # row: source, col: target
        
        deg = degree(row, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x, norm=norm)
        
    def message(self, x_j, norm):
        # x_j has shape: (E, out_channels)
        return norm.view(-1, 1) * x_j
    
    def update(self, aggr_out):
        # aggr_out has shape (N, out_channels)
        return aggr_out
        

In [29]:
edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [2, 1],
                           [0, 3],
                           [3, 2]], dtype=torch.long)
conv = GCNConv(24, 32)
x = conv(x.t(), edge_index.t().contiguous())

#### Custom Dataset
---

No explicit examples, just grab a dataset and follow [this](https://pytorch-geometric.readthedocs.io/en/latest/notes/batching.html).